In [ ]:
# 작업 흐름

# ../data 내에 소스별 qa.csv, mcqa.csv 파일 로드
# ../../models/name_of_llm 폴더에 있는 모델 로드
# 4bit 양자화 후 파인튜닝
# 파인튜닝 완료 후 모델 저장

In [ ]:
# 필요한 패키지 설치
%pip install -q pandas tqdm 
%pip install -q transformers==4.55.0 # llm requires >=4.46.0
%pip install -q safetensors==0.4.3 # downgrade for torch 2.1.0
%pip install -q bitsandbytes==0.43.2 accelerate==1.9.0 # quantization
%pip install -q datasets peft scikit-learn # 추가 필요 패키지

print("✅ 패키지 설치 완료!")

# QLoRA 파인튜닝을 위한 라이브러리 임포트
import os
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import warnings
warnings.filterwarnings('ignore')

# GPU 메모리 확인
print(f"🔥 CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🔥 GPU 개수: {torch.cuda.device_count()}")
    print(f"🔥 현재 GPU: {torch.cuda.current_device()}")
    print(f"🔥 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")


In [ ]:
# 데이터 로딩 및 전처리
def load_and_combine_datasets():
    """모든 소스의 qa.csv와 mcqa.csv 파일을 로딩하고 결합"""
    
    data_sources = [
        'FinShibainu',
        'SecBench', 
        'CyberMetric'
    ]
    
    all_qa_data = []
    all_mcqa_data = []
    
    print("📂 데이터 로딩 중...")
    
    for source in data_sources:
        # QA 데이터 로딩
        qa_path = f"../data/{source}/qa.csv" if source != 'SecBench' else f"../data/{source}/qa_org.csv"
        if os.path.exists(qa_path):
            qa_df = pd.read_csv(qa_path, encoding='utf-8-sig')
            qa_df['source'] = source
            all_qa_data.append(qa_df)
            print(f"✅ {source} QA 데이터: {len(qa_df)}개")
        else:
            print(f"❌ {qa_path} 파일을 찾을 수 없습니다.")
            
        # MCQA 데이터 로딩  
        mcqa_path = f"../data/{source}/mcqa.csv" if source != 'SecBench' and source != 'CyberMetric' else f"../data/{source}/mcqa_org.csv"
        if os.path.exists(mcqa_path):
            mcqa_df = pd.read_csv(mcqa_path, encoding='utf-8-sig')
            mcqa_df['source'] = source
            all_mcqa_data.append(mcqa_df)
            print(f"✅ {source} MCQA 데이터: {len(mcqa_df)}개")
        else:
            print(f"❌ {mcqa_path} 파일을 찾을 수 없습니다.")
    
    # 데이터 결합
    combined_qa = pd.concat(all_qa_data, ignore_index=True) if all_qa_data else pd.DataFrame()
    combined_mcqa = pd.concat(all_mcqa_data, ignore_index=True) if all_mcqa_data else pd.DataFrame()
    
    print(f"\n📊 전체 QA 데이터: {len(combined_qa)}개")
    print(f"📊 전체 MCQA 데이터: {len(combined_mcqa)}개")
    
    return combined_qa, combined_mcqa

# 데이터 로딩
qa_data, mcqa_data = load_and_combine_datasets()


In [ ]:
# 데이터 전처리 및 포맷팅
def format_data_for_training(qa_data, mcqa_data):
    """학습용 데이터 포맷팅 - 대화형 형태로 변환"""
    
    formatted_data = []
    
    # QA 데이터 처리
    if not qa_data.empty:
        print("🔄 QA 데이터 포맷팅 중...")
        for _, row in qa_data.iterrows():
            question = row['Question']
            answer = row['Answer']
            
            # 대화형 포맷으로 변환
            text = f"질문: {question}\n답변: {answer}"
            formatted_data.append({"text": text, "type": "qa", "source": row.get('source', 'unknown')})
    
    # MCQA 데이터 처리  
    if not mcqa_data.empty:
        print("🔄 MCQA 데이터 포맷팅 중...")
        for _, row in mcqa_data.iterrows():
            question = row['Question']
            answer = row['Answer']
            
            # 대화형 포맷으로 변환
            text = f"질문: {question}\n답변: {answer}"
            formatted_data.append({"text": text, "type": "mcqa", "source": row.get('source', 'unknown')})
    
    print(f"📝 총 {len(formatted_data)}개의 학습 샘플 생성")
    
    # 샘플 확인
    if formatted_data:
        print("\n🔍 샘플 데이터:")
        print(formatted_data[0]['text'][:200] + "...")
    
    return formatted_data

# 데이터 포맷팅
formatted_data = format_data_for_training(qa_data, mcqa_data)

# Hugging Face Dataset으로 변환
from sklearn.model_selection import train_test_split

if formatted_data:
    # 훈련/검증 데이터 분할
    train_data, val_data = train_test_split(formatted_data, test_size=0.1, random_state=42)
    
    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)
    
    dataset = DatasetDict({
        'train': train_dataset,
        'validation': val_dataset
    })
    
    print(f"🎯 훈련 데이터: {len(train_dataset)}개")
    print(f"🎯 검증 데이터: {len(val_dataset)}개")
else:
    print("❌ 포맷팅된 데이터가 없습니다.")


In [ ]:
# 모델 선택 및 설정
AVAILABLE_MODELS = {
    "gemma-ko-7b": "beomi/gemma-ko-7b",
    "ax-4.0-light-7b": "axolotl-ai-co/ax-4.0-light-7b", 
    "midm-2.0-11.5b": "microsoft/DialoGPT-medium-2.0-11.5b"  # 실제 모델명으로 수정 필요
}

# 사용할 모델 선택 (여기서 변경하세요)
SELECTED_MODEL = "gemma-ko-7b"  # "gemma-ko-7b", "ax-4.0-light-7b", "midm-2.0-11.5b" 중 선택

MODEL_NAME = AVAILABLE_MODELS[SELECTED_MODEL]
MODEL_PATH = f"../../models/{SELECTED_MODEL}"  # 로컬 모델 경로

print(f"🎯 선택된 모델: {SELECTED_MODEL}")
print(f"🤖 모델명: {MODEL_NAME}")
print(f"📁 로컬 경로: {MODEL_PATH}")

# 4bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)



# 토크나이저 로딩
try:
    if os.path.exists(MODEL_PATH):
        tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
        print("✅ 로컬 토크나이저 로딩 완료")
    else:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
        print("✅ 허깅페이스 토크나이저 로딩 완료")
        
    # 패딩 토큰 설정
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
except Exception as e:
    print(f"❌ 토크나이저 로딩 실패: {e}")

# 모델 로딩
try:
    if os.path.exists(MODEL_PATH):
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=torch.bfloat16
        )
        print("✅ 로컬 모델 로딩 완료")
    else:
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            quantization_config=bnb_config,
            device_map="auto", 
            trust_remote_code=True,
            torch_dtype=torch.bfloat16
        )
        print("✅ 허깅페이스 모델 로딩 완료")
        
    print(f"🔥 모델이 로딩된 디바이스: {model.device}")
    
except Exception as e:
    print(f"❌ 모델 로딩 실패: {e}")


In [ ]:
# 모델별 LoRA target_modules 설정
TARGET_MODULES_CONFIG = {
    "gemma-ko-7b": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    "ax-4.0-light-7b": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], 
    "midm-2.0-11.5b": ["q_proj", "k_proj", "v_proj", "o_proj"]  # 모델 구조에 따라 조정
}

# LoRA 설정 및 모델 준비
lora_config = LoraConfig(
    r=16,                    # LoRA rank
    lora_alpha=32,           # LoRA scaling parameter
    target_modules=TARGET_MODULES_CONFIG.get(SELECTED_MODEL, ["q_proj", "k_proj", "v_proj", "o_proj"]),
    lora_dropout=0.1,        # LoRA dropout
    bias="none",
    task_type="CAUSAL_LM",
)

print("🔧 LoRA 설정:")
print(f"  - Rank: {lora_config.r}")
print(f"  - Alpha: {lora_config.lora_alpha}")
print(f"  - Dropout: {lora_config.lora_dropout}")
print(f"  - Target modules: {lora_config.target_modules}")

# 모델을 kbit 학습용으로 준비
try:
    model = prepare_model_for_kbit_training(model)
    print("✅ kbit 학습 준비 완료")
    
    # LoRA 어댑터 적용
    model = get_peft_model(model, lora_config)
    print("✅ LoRA 어댑터 적용 완료")
    
    # 학습 가능한 파라미터 확인
    model.print_trainable_parameters()
    
except Exception as e:
    print(f"❌ LoRA 설정 실패: {e}")


In [ ]:
# 데이터 토크나이징
def tokenize_function(examples):
    """데이터를 토크나이징하는 함수"""
    # 텍스트 토크나이징
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding=False,  # Dynamic padding 사용
        max_length=1024,
        return_tensors=None
    )
    
    # labels는 input_ids와 동일하게 설정 (language modeling)
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

# 데이터셋 토크나이징
if 'dataset' in locals():
    print("🔤 데이터 토크나이징 중...")
    
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset["train"].column_names,
        desc="토크나이징 진행"
    )
    
    print("✅ 토크나이징 완료")
    print(f"📊 토크나이징된 훈련 데이터: {len(tokenized_dataset['train'])}개")
    print(f"📊 토크나이징된 검증 데이터: {len(tokenized_dataset['validation'])}개")
    
    # 샘플 확인
    sample = tokenized_dataset["train"][0]
    print(f"🔍 샘플 토큰 길이: {len(sample['input_ids'])}")
    print(f"🔍 샘플 디코딩: {tokenizer.decode(sample['input_ids'][:100])}...")
else:
    print("❌ 데이터셋이 준비되지 않았습니다.")


In [ ]:
# 훈련 설정 - 모델별 출력 디렉토리
OUTPUT_DIR = f"./qlora-finetuned-{SELECTED_MODEL}"
CHECKPOINT_DIR = f"./checkpoints-{SELECTED_MODEL}"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # 훈련 파라미터
    num_train_epochs=3,              # 에포크 수
    per_device_train_batch_size=1,   # 배치 크기 (메모리에 따라 조정)
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,   # 그래디언트 누적 (실질적 배치 크기 = 1*8=8)
    
    # 학습률 및 스케줄러
    learning_rate=2e-4,              # 학습률
    warmup_ratio=0.1,                # 워밍업 비율
    lr_scheduler_type="cosine",      # 학습률 스케줄러
    
    # 정규화
    weight_decay=0.01,               # 가중치 감소
    max_grad_norm=1.0,               # 그래디언트 클리핑
    
    # 로깅 및 저장
    logging_steps=10,                # 로깅 주기
    save_steps=100,                  # 체크포인트 저장 주기
    eval_steps=100,                  # 평가 주기
    save_total_limit=3,              # 저장할 체크포인트 최대 개수
    
    # 평가 설정
    evaluation_strategy="steps",     # 평가 전략
    load_best_model_at_end=True,     # 최고 성능 모델 로딩
    metric_for_best_model="eval_loss", # 최고 성능 기준
    greater_is_better=False,         # 낮을수록 좋음
    
    # 메모리 최적화
    dataloader_pin_memory=False,     # 메모리 핀 비활성화
    remove_unused_columns=False,     # 사용하지 않는 컬럼 제거 비활성화
    
    # 기타
    report_to=None,                  # wandb 등 비활성화
    run_name="qlora-finetune",       # 실행 이름
    seed=42,                         # 시드
)

print("⚙️ 훈련 설정:")
print(f"  - 에포크: {training_args.num_train_epochs}")
print(f"  - 배치 크기: {training_args.per_device_train_batch_size}")
print(f"  - 그래디언트 누적: {training_args.gradient_accumulation_steps}")
print(f"  - 실질적 배치 크기: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - 학습률: {training_args.learning_rate}")
print(f"  - 출력 디렉토리: {OUTPUT_DIR}")

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM이므로 False
    pad_to_multiple_of=8  # 효율성을 위한 패딩
)

print("✅ 데이터 콜레이터 설정 완료")


In [ ]:
# 트레이너 생성 및 훈련 실행
if 'tokenized_dataset' in locals() and 'model' in locals():
    print("🚀 트레이너 생성 중...")
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        data_collator=data_collator,
        tokenizer=tokenizer,
    )
    
    print("✅ 트레이너 생성 완료")
    print(f"📊 훈련 샘플 수: {len(tokenized_dataset['train'])}")
    print(f"📊 검증 샘플 수: {len(tokenized_dataset['validation'])}")
    
    # 훈련 시작 전 메모리 상태 확인
    if torch.cuda.is_available():
        print(f"🔥 훈련 전 GPU 메모리: {torch.cuda.memory_allocated()/1024**3:.2f}GB / {torch.cuda.memory_reserved()/1024**3:.2f}GB")
    
    print("\n🎯 파인튜닝 시작...")
    print("=" * 50)
    
    try:
        # 훈련 실행
        trainer.train()
        print("\n✅ 파인튜닝 완료!")
        
        # 훈련 후 메모리 상태
        if torch.cuda.is_available():
            print(f"🔥 훈련 후 GPU 메모리: {torch.cuda.memory_allocated()/1024**3:.2f}GB / {torch.cuda.memory_reserved()/1024**3:.2f}GB")
        
    except Exception as e:
        print(f"❌ 훈련 중 오류 발생: {e}")
        print("💡 배치 크기를 줄이거나 그래디언트 누적 단계를 늘려보세요.")
        
else:
    print("❌ 모델 또는 데이터셋이 준비되지 않았습니다.")
    print("이전 셀들을 먼저 실행해주세요.")


In [ ]:
# 모델 저장 및 최종 정리
if 'trainer' in locals():
    print("💾 모델 저장 중...")
    
    try:
        # 최종 모델 저장
        trainer.save_model(OUTPUT_DIR)
        tokenizer.save_pretrained(OUTPUT_DIR)
        
        print(f"✅ 모델 저장 완료: {OUTPUT_DIR}")
        
        # LoRA 어댑터만 별도 저장
        lora_adapter_dir = f"./lora-adapter-{SELECTED_MODEL}"
        model.save_pretrained(lora_adapter_dir)
        print(f"✅ LoRA 어댑터 저장 완료: {lora_adapter_dir}")
        
        # 훈련 로그 저장
        if hasattr(trainer, 'state') and trainer.state.log_history:
            import json
            with open(f"{OUTPUT_DIR}/training_log.json", 'w', encoding='utf-8') as f:
                json.dump(trainer.state.log_history, f, indent=2, ensure_ascii=False)
            print("✅ 훈련 로그 저장 완료")
        
    except Exception as e:
        print(f"❌ 모델 저장 실패: {e}")
    
    # 메모리 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("🧹 GPU 메모리 정리 완료")
        
    print("\n🎉 QLoRA 파인튜닝 전체 과정 완료!")
    print("=" * 50)
    print(f"📁 저장된 파일들:")
    print(f"  - 파인튜닝된 모델: {OUTPUT_DIR}")
    print(f"  - LoRA 어댑터: {lora_adapter_dir}")
    print(f"  - 체크포인트: {CHECKPOINT_DIR}")
    
else:
    print("❌ 트레이너가 생성되지 않았습니다.")
    print("이전 셀들을 먼저 실행해주세요.")


# 🎯 QLoRA 파인튜닝 가이드

## 📋 실행 순서
1. **라이브러리 설치**: 필요한 패키지들이 설치되어 있는지 확인
2. **데이터 로딩**: 모든 소스의 qa.csv, mcqa.csv 파일 로딩
3. **모델 준비**: 4bit 양자화 및 LoRA 설정
4. **훈련 실행**: 파인튜닝 진행
5. **모델 저장**: 결과 모델 저장

## ⚙️ 설정 조정 가이드

### 메모리 부족 시:
- `per_device_train_batch_size`: 1 → 1 (이미 최소)
- `gradient_accumulation_steps`: 8 → 16 (실질적 배치 크기 유지)
- `max_length`: 1024 → 512 (토큰 길이 줄이기)

### 훈련 속도 향상:
- `num_train_epochs`: 3 → 1 (에포크 줄이기)
- `eval_steps`: 100 → 500 (평가 빈도 줄이기)
- `save_steps`: 100 → 500 (저장 빈도 줄이기)

### LoRA 설정 조정:
- `r`: 16 → 8 (파라미터 수 줄이기)
- `lora_alpha`: 32 → 16 (스케일링 조정)

## 🔧 패키지 설치
첫 번째 셀에서 자동으로 설치됩니다:
- `transformers==4.55.0`: LLM 라이브러리
- `safetensors==0.4.3`: 모델 저장 형식
- `bitsandbytes==0.43.2`: 양자화 라이브러리
- `accelerate==1.9.0`: 분산 학습 지원
- `datasets`, `peft`, `scikit-learn`: 데이터셋 및 LoRA

## 🤖 지원 모델들
- **gemma-ko-7b**: 한국어에 특화된 Gemma 모델
- **ax-4.0-light-7b**: Axolotl 경량화 모델  
- **midm-2.0-11.5b**: Microsoft DialoGPT 기반 모델

## 📊 예상 결과
- **파인튜닝된 모델**: `./qlora-finetuned-{model_name}/`
- **LoRA 어댑터**: `./lora-adapter-{model_name}/`
- **체크포인트**: `./checkpoints-{model_name}/`
- **훈련 로그**: `training_log.json`

## 🔄 모델 변경 방법
셀 4에서 `SELECTED_MODEL` 변수를 변경하세요:
```python
SELECTED_MODEL = "gemma-ko-7b"     # 또는 "ax-4.0-light-7b", "midm-2.0-11.5b"
```


In [ ]:
# LoRA 어댑터를 원본 모델에 병합하여 BF16 모델 생성
def merge_and_save_bf16_model():
    """LoRA 어댑터를 원본 모델에 병합하고 BF16으로 저장"""
    
    print("🔄 LoRA 어댑터를 원본 모델에 병합 중...")
    
    try:
        # 원본 모델을 BF16으로 다시 로딩 (양자화 없이)
        base_model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH if os.path.exists(MODEL_PATH) else MODEL_NAME,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )
        print("✅ 원본 모델 BF16 로딩 완료")
        
        # LoRA 어댑터 로딩 및 병합
        from peft import PeftModel
        
        lora_adapter_dir = f"./lora-adapter-{SELECTED_MODEL}"
        if os.path.exists(lora_adapter_dir):
            # LoRA 어댑터를 원본 모델에 로딩
            model_with_lora = PeftModel.from_pretrained(base_model, lora_adapter_dir)
            print("✅ LoRA 어댑터 로딩 완료")
            
            # LoRA를 원본 모델에 병합
            merged_model = model_with_lora.merge_and_unload()
            print("✅ LoRA 어댑터 병합 완료")
            
            # BF16 모델 저장
            bf16_output_dir = f"./merged-bf16-{SELECTED_MODEL}"
            merged_model.save_pretrained(
                bf16_output_dir,
                torch_dtype=torch.bfloat16,
                safe_serialization=True
            )
            
            # 토크나이저도 함께 저장
            tokenizer.save_pretrained(bf16_output_dir)
            
            print(f"✅ BF16 병합 모델 저장 완료: {bf16_output_dir}")
            
            # 메모리 정리
            del base_model, model_with_lora, merged_model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            return bf16_output_dir
            
        else:
            print(f"❌ LoRA 어댑터를 찾을 수 없습니다: {lora_adapter_dir}")
            return None
            
    except Exception as e:
        print(f"❌ BF16 모델 병합 실패: {e}")
        return None

# BF16 모델 생성 실행
if 'trainer' in locals() and os.path.exists(f"./lora-adapter-{SELECTED_MODEL}"):
    print("\n🔄 LoRA 어댑터를 BF16 모델로 병합 중...")
    print("=" * 50)
    
    bf16_model_path = merge_and_save_bf16_model()
    
    if bf16_model_path:
        print(f"\n🎉 BF16 병합 모델 생성 완료!")
        print(f"📁 저장 위치: {bf16_model_path}")
        print("💡 이 모델은 양자화 없이 직접 사용할 수 있습니다.")
    else:
        print("❌ BF16 모델 생성 실패")
        
else:
    print("❌ 훈련이 완료되지 않았거나 LoRA 어댑터가 없습니다.")
    print("이전 셀들을 먼저 실행해주세요.")
